##2. Search Engine

I have put my custom functions in functions.py and the search engine function in searchEngine.py and I am importing them in the beginning here:

In [ ]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
from  tqdm import tqdm
import time
import re
import csv
import os
import pandas as pd
import functions as f
from functions import extract_fees
from functions import preprocess_text
from functions import get__currency_rates_api
from functions import convert_currency
from searchEngine import conjunction_search

Before starting with the question, I have loaded all the 6000 TSV files into one dataframe using my TSV_to_dataframe function in the functions.py:

In [ ]:
column_names = [
    "courseName",
    "universityName",
    "facultyName",
    "isItFullTime",
    "description",
    "startDate",
    "fees",
    "modality",
    "duration",
    "city",
    "country",
    "administration",
    "url"
]

folder_name = 'folderTSV'
num_files = 6000

courses_df = f.TSV_to_dataframe(column_names, folder_name, num_files)

In [ ]:
courses_df.shape[0]

6000

In [ ]:
courses_df.head(3)

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,ProcessedDescription,currency,fees (EUR)
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role in ...,September,NaN,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,3d visualis anim play role mani area popular m...,None,NaN
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound finan...,September,34750.0,MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,busi govern reli sound financi knowledg underp...,£,39700.67
2,"Accounting, Accountability & Financial Managem...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial Man...",September,NaN,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...,account account financi manag msc cours provi...,None,NaN


###2.0 Preprocessing

2.0.0 Preprocessing the text

First, you must pre-process all the information collected for each MSc by:

Removing stopwords
Removing punctuation
Stemming
Anything else you think it's needed
For this purpose, you can use the `nltk library.


I have built a function preprocess_text in the functions.py using nltk corpus stopwords and Porter stemmer. I am applying the function over each rows of the dataframe here and keeping the processed text of the description column in a new column in the dataframe:

In [ ]:
# Preprocess the 'description' column
courses_df['ProcessedDescription'] = courses_df['description'].apply(preprocess_text)

In [ ]:
courses_df.loc[10,['description','ProcessedDescription']]

description             The Analytical Toxicology MSc is a unique stud...
ProcessedDescription    analyt toxicolog msc uniqu studi cours integr ...
Name: 10, dtype: object

###2.0.1) Preprocessing the fees column
Moreover, we want the field fees to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a float column renamed fees (CHOSEN COMMON CURRENCY).

In [ ]:
courses_df[courses_df['fees']!=''][['fees']].head(5)

,fees
1,"UK: £18,000 (Total)International: £34,750 (Total)"
5,"UK: £13,750 (Total)International: £31,000 (Total)"
7,Tuition fee per year (non-EU/EEA students): 15...
8,Tuition fee per year (non-EU/EEA students): 15...
9,"UK: £12,500 (Total)International: £28,750 (Total)"


Initially, I am extracting the numerical value of fees and keeping them in the fees column and creating a currency column to keep the currency symbol, to understand which currency fees is present in the row. I am doing all of this inside the extract_fees function and that I am applying to each row in the dataframe:

In [ ]:
courses_df[['fees', 'currency']] = courses_df['fees'].apply(extract_fees).apply(pd.Series)

In [ ]:
courses_df[courses_df['fees'].notna()][['fees','currency']].head(5)

,fees,currency
1,34750.0,£
5,31000.0,£
7,15000.0,€
8,15000.0,€
9,28750.0,£


Now that I have the numeric values of the fees and the currency in the fees and currency column,I can convert the fees to a common currency. I am choosing Euro to be the common uniform currency for my case.

My Idea- I am using the exchange-rates.com API in the function get__currency_rates_api to get the currency rates in terms of Euros in a dictionary which is kept inside exchange_rates. Since, the API wants me to insert the currency that I want to convert, but for that purpose, I will need to make 6000 different api calls if apply that to each individual row and my API key has a request limit for free access. So I am just inserting Euros and getting the inverted rates in each currencies as a dictionary. Thus, in the convert_currency function, I am dividing the amount by the rate (essentially the inverted rate) to get the value in my chosen common currency. In this way, I am just making one request for this API and getting my work of conversion done applying to each row of the dataframe. At the end, am keeping that in the fees(EUR) column.

In [ ]:
common_currency= 'EUR'
exchange_rates=get__currency_rates_api(common_currency)

In [ ]:
# Apply the conversion to the 'fees' column in the DataFrame
courses_df['fees (EUR)'] = courses_df.apply(lambda row: convert_currency(exchange_rates, row['fees'], row['currency']), axis=1)


In [ ]:
courses_df[courses_df['fees'].notna()][['fees','fees (EUR)','currency']].head(5)

,fees,fees (EUR),currency
1,34750.0,39700.67,£
5,31000.0,35416.43,£
7,15000.0,15000.00,€
8,15000.0,15000.00,€
9,28750.0,32845.88,£


As I have processed the dataframe to an extent, it is a safe idea to store the entire dataframe in a single TSV file so that, later on, I won't need to run the same code to make the dataframe but I can load it from the TSV file using pandas library.

In [ ]:
tsv_file_path = 'courses_data_processed.tsv'
courses_df.to_csv(tsv_file_path, sep='\t', index=False)

###2.1. Conjunctive query
For the first version of the search engine, we narrowed our interest to the description of each course. It means that you will evaluate queries only concerning the course's description.

####2.1.1) Create your index!
Before building the index,

Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).
Then, the first brick of your homework is to create the Inverted Index.

**Creating Vocabulary and saving it as vocabulary.txt**

In [ ]:
# Create a set to store unique terms
unique_terms = set()

# Iterate through the DataFrame to collect unique terms
for index, row in courses_df.iterrows():
    stemmed_words = row["ProcessedDescription"].split()
    unique_terms.update(stemmed_words)

# Create a vocabulary by assigning term_ids to unique terms
vocabulary = {term: idx for idx, term in enumerate(unique_terms, start=1)}

# Save the vocabulary to a text file
with open("vocabulary.txt", "w") as vocab_file:
    for term, term_id in vocabulary.items():
        vocab_file.write(f"{term} {term_id}\n")


**Creating Inverted Index**

In [ ]:
from collections import defaultdict
# Initialize an empty inverted index
inverted_index = defaultdict(list)

# Iterate through the DataFrame rows and update the inverted index
for index, row in courses_df.iterrows():
    stemmed_words = row["ProcessedDescription"].split()
    for term in stemmed_words:
        term_id = vocabulary.get(term)  # Get the term_id from the vocabulary
        if term_id is not None:
            inverted_index[term_id].append(index)

# Convert the defaultdict to a regular dictionary
inverted_index = dict(inverted_index)

In [ ]:
len(inverted_index)

7307

**Saving the Inverted Index as inverted_index.json**

In [ ]:
# Save the inverted index to a JSON file
import json
with open("inverted_index.json", "w") as index_file:
    json.dump(inverted_index, index_file)

####2.1.2) Execute the query
Given a query input by the user, for example:

advanced knowledge

**Search Engine 1.0**

Loading the dataframe I had saved in the tsv file already:

In [ ]:
courses_df=pd.read_csv('courses_data_processed.tsv',sep='\t')

Loading the vocabulary and inverted_index that I saved before:

In [ ]:
import json

# Load vocabulary from "vocabulary.txt"
vocabulary = {}
with open("vocabulary.txt", "r") as vocab_file:
    for line in vocab_file:
        term, term_id = line.strip().split()
        vocabulary[term] = int(term_id)

# Load inverted index from "inverted_index.json"
with open("inverted_index.json", "r") as index_file:
    inverted_index = json.load(index_file)


*In the searchEngine.py I have built the function 'conjunction_search'. The function takes the dataframe, vocabulary, inverted_index and the query as parameters. It processes the query using nltk library and then checks the presence of every word in the query using the vocabulary and inverted index and then returns the index values which have the query words conjunctively. At the end, I printed the result_df with the necessary information required.*

In [ ]:
query = "advanced knowledge"

# Search for the query and get the matching DataFrame
result_df = conjunction_search(courses_df, vocabulary, inverted_index, query)

# Print the result DataFrame
result_df[['courseName','universityName','description','url']]

,courseName,universityName,description,url
4099,Global Meetings and Events Management MSc / PGDip,University College Birmingham,Become part of an events industry worth an est...,https://www.findamasters.com/masters-degrees/c...
2568,Dance Science MSc,University of Chichester,This suite of MSc programmes is designed for s...,https://www.findamasters.com/masters-degrees/c...
2569,Data Analysis for Business Intelligence - MSc,University of Leicester,The course is designed for students who want t...,https://www.findamasters.com/masters-degrees/c...
10,Analytical Toxicology MSc,King’s College London,The Analytical Toxicology MSc is a unique stud...,https://www.findamasters.com/masters-degrees/c...
522,Accounting and Finance - MSc,University of Sussex,On this MSc you’ll advance your accounting and...,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...
2540,Cyber Security MSc,Keele University,IT systems are a vital part of every company's...,https://www.findamasters.com/masters-degrees/c...
4590,Intelligent Transport Planning and Engineering...,University of East London,The programme is designed to meet the increasi...,https://www.findamasters.com/masters-degrees/c...
495,(MSc/PGDip/PGCert) - Advanced Clinical Practic...,University of Warwick,Imagine if you had the confidence to react mor...,https://www.findamasters.com/masters-degrees/c...
2037,Clinical Pharmacy - MSc,University of Sunderland,The Clinical Pharmacy MSc has been designed to...,https://www.findamasters.com/masters-degrees/c...


In [ ]:
print(result_df.loc[10,'description'])

The Analytical Toxicology MSc is a unique study course that integrates theoretical and practical aspects of analytical science with clinical and forensic toxicology. This course will provide you with a detailed knowledge and comprehensive understanding of advanced analytical toxicology and its applications.
